# Connecting Google Drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# First Downloading Packages

In [ ]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 7.4 MB/s 
     |████████████████████████████████| 1.2 MB 72.5 MB/s 
     |████████████████████████████████| 1.8 MB 57.4 MB/s 
     |████████████████████████████████| 9.1 MB 54.2 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 4.4 MB 59.7 MB/s 
     |████████████████████████████████| 362 kB 73.0 MB/s 
     |████████████████████████████████| 6.6 MB 54.4 MB/s 
     |████████████████████████████████| 86 kB 6.6 MB/s 
     |████████████████████████████████| 596 kB 72.3 MB/s 
     |████████████████████████████████| 181 kB 53.7 MB/s 
     |████████████████████████████████| 145 kB 72.3 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 140 kB 73.9 MB/s 
     |████████████████████████████████| 212 kB 73.1 MB/s 
     |████████████████████████████████

# 1.1 Locate Drive


In [8]:
import os 
os.chdir("/content/drive/MyDrive/AraElectra") #place that have data and reader script

# 1.2 Download The Competition scripts

In [9]:
!git clone https://gitlab.com/bigirqu/quranqa.git


fatal: destination path 'quranqa' already exists and is not an empty directory.


In [10]:
!pip install -r quranqa/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 1.3 Imports

In [12]:
import json
import pandas as pd

#Second:Preprocessing Download

##Downloading and Setting up AraBERT

In [11]:
!git clone https://github.com/aub-mind/arabert
!pip install pyarabic
from arabert.preprocess import ArabertPreprocessor
prep = ArabertPreprocessor("bert-base-arabert") #or empty string it's the same 

fatal: destination path 'arabert' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[2022-06-19 14:28:28,745 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


# Third: Reading Data
-our data shape is list of dictioneries

In [ ]:
import read_write_qrcd as qrcd
# train=qrcd.load_jsonl("/content/drive/MyDrive/AraElectra/quranqa/datasets/qrcd_v1.1_train.jsonl")
train=qrcd.load_jsonl("/content/drive/MyDrive/GP-2022 Models Backup/train_and_augmented.jsonl")

Loaded 1420 records from /content/drive/MyDrive/GP-2022 Models Backup/train_and_augmented.jsonl


In [ ]:
val=qrcd.load_jsonl("/content/drive/MyDrive/AraElectra/quranqa/datasets/qrcd_v1.1_dev.jsonl")

Loaded 109 records from /content/drive/MyDrive/AraElectra/quranqa/datasets/qrcd_v1.1_dev.jsonl


In [ ]:
test=qrcd.load_jsonl("/content/drive/MyDrive/AraElectra/quranqa/datasets/qrcd_v1.1_test_noAnswers.jsonl")
#test[:7]

Loaded 274 records from /content/drive/MyDrive/AraElectra/quranqa/datasets/qrcd_v1.1_test_noAnswers.jsonl


# Fourth: Preparing Data to Enter The Model
1.   adjusting train and test data format  
2.   applying arabert preprocessing on train data questions only

## Adjusting Data Formats

In [ ]:
def prepare_data_arabert(data):
  data_prepared=[]
  answers=[] #list for list of answers for every record
  for x in data: #for record in data set
    for i in range(len(x['answers'])):#loop for records that have more than one answer
            
      answers.append({
                             'text': x['answers'][i]['text'],
                          'answer_start': x['answers'][i]['start_char']
                          })
                     
    
    dict={'context':x['passage'],
        "qas": [
            {
                'id': x['pq_id'],
            "is_impossible": False,
            'question': x['question'],
            'answers': answers #list of answers we have done in the second for loop
            },
         ],}
    if(answers==[]):
      dict["qas"][0]['is_impossible']=True
    data_prepared.append(dict)#adding record to the new list of dictioneries (data)
    answers=[] #empty list of answers for next record answers
  
  return data_prepared


In [ ]:
def prepare_data_arabert_arabert(data):
  data_prepared=[]
  answers=[] #list for list of answers for every record
  for x in data: #for record in data set
    for i in range(len(x['answers'])):#loop for records that have more than one answer
            
      answers.append({
                             'text': x['answers'][i]['text'],
                          'answer_start': x['answers'][i]['start_char']
                          })
                     
    
    dict={'context':x['passage'],
        "qas": [
            {
                'id': x['pq_id'],#+"_"+str(i+1), #adding number of answer to id
            "is_impossible": False,
            'question': prep.preprocess(x['question']), #applying arabert preprocess on question
            'answers': answers #list of answers we have done in the second for loop
            },
         ],}
    data_prepared.append(dict) #adding record to the new list of dictioneries (data)
    answers=[] #empty list of answers for next record answers
  
  return data_prepared

##Applying AraBERT Preprocessing on train data questions only

In [ ]:
#train=prepare_data_araelectra_camel(train)
#train=prepare_data_araelectra(train)
train=prepare_data_arabert_arabert(train)
val=prepare_data_arabert(val)
#val=prepare_data_araelectra_arabert(val)

In [ ]:
test=prepare_data_arabert(test)

# Fifth: The Model Preparation and Training

In [ ]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [ ]:
model_type = "bert"
model_name = "aubmindlab/bert-base-arabertv2"

## Model Arguments

In [ ]:
#help(QuestionAnsweringArgs)

In [ ]:
# Configure the model 
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 64
model_args.eval_batch_size=32
model_args.evaluate_during_training = True
model_args.n_best_size=5
model_args.num_train_epochs=25
model_args.max_seq_length=196
model_args.wandb_project="Arabertv2 QA"
#model_args.wandb_kwargs= {"name": model_name}
model_args.use_cached_eval_features= True
model_args.overwrite_output_dir= True
model_args.output_dir= f"outputs/{model_type}"
model_args.best_model_dir= f"outputs/{model_type}/best_model"
model_args.save_model_every_epoch= False
model_args.save_eval_checkpoints= False

## Clean Cache 

In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14780765484351376073
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15451553792
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16099361686388688215
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
 xla_global_id: 416903419]

## Intialize The Model

In [ ]:
model = QuestionAnsweringModel(
    model_type,model_name, args=model_args ,use_cuda=True
)

In [ ]:
### Remove output folder
!rm -rf outputs

In [ ]:
#checking cache 
!nvidia-smi 

Sat Jun 18 20:18:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    35W / 250W |   1045MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Train The Model

In [ ]:
# Train the model
model.train_model(train, eval_data=val)

add example index and unique id: 100%|██████████| 1420/1420 [00:00<00:00, 384475.61it/s]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/45 [00:00<?, ?it/s]

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

Running Epoch 43 of 45:   0%|          | 0/23 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 44 of 45:   0%|          | 0/23 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/4 [00:00<?, ?it/s]

(1035,
 {'correct': [0,
   3,
   4,
   8,
   15,
   16,
   18,
   18,
   19,
   20,
   17,
   19,
   19,
   19,
   20,
   20,
   20,
   18,
   19,
   20,
   19,
   20,
   21,
   20,
   20,
   20,
   20,
   19,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   19,
   19,
   19,
   18,
   19,
   20,
   20,
   20],
  'eval_loss': [-0.44635009765625,
   -1.297607421875,
   -2.28173828125,
   -3.19189453125,
   -3.59228515625,
   -3.83154296875,
   -4.0732421875,
   -4.3603515625,
   -4.5068359375,
   -4.6669921875,
   -4.736328125,
   -4.7265625,
   -4.8134765625,
   -4.8876953125,
   -4.94140625,
   -5.0224609375,
   -5.1123046875,
   -5.068359375,
   -5.1552734375,
   -5.3115234375,
   -5.3486328125,
   -5.3662109375,
   -5.46875,
   -5.4423828125,
   -5.455078125,
   -5.4677734375,
   -5.5615234375,
   -5.6015625,
   -5.650390625,
   -5.6982421875,
   -5.7099609375,
   -5.7041015625,
   -5.70703125,
   -5.728515625,
   -5.7890625,
   -5.7841796875,
   -5.779296875,
   -5

# Sixth Evaluating The Model

## Evaluating The Model

In [ ]:
import sklearn
result, texts = model.eval_model(val,acc=sklearn.metrics.accuracy_score)

Running Evaluation:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
#texts['correct_text']

In [ ]:
result

{'correct': 20,
 'similar': 48,
 'incorrect': 41,
 'acc': 0.1834862385321101,
 'eval_loss': -5.765625}

## Make Predictions With The Model

In [ ]:
def Predict(data):
  to_predict = []
  for x in data:
    to_predict.append({
        "context": x['context'],#'وإن نكثوا أيمانهم من بعد عهدهم وطعنوا في دينكم فقاتلوا أئمة الكفر إنهم لا أيمان لهم لعلهم ينتهون. ألا تقاتلون قوما نكثوا أيمانهم وهموا بإخراج الرسول وهم بدءوكم أول مرة أتخشونهم فالله أحق أن تخشوه إن كنتم مؤمنين. قاتلوهم يعذبهم الله بأيديكم ويخزهم وينصركم عليهم ويشف صدور قوم مؤمنين. ويذهب غيظ قلوبهم ويتوب الله على من يشاء والله عليم حكيم. أم حسبتم أن تتركوا ولما يعلم الله الذين جاهدوا منكم ولم يتخذوا من دون الله ولا رسوله ولا المؤمنين وليجة والله خبير بما تعملون.',
        "qas": [
                {
                    "question": prep.preprocess(x['qas'][0]['question']),#"متى يحل الإسلام دم الشخص؟",#AraBERT
                    #"question":x['qas'][0]['question'],
                    "id": x['qas'][0]['id'],
                }
                ],
                })
  return data

             
      


In [ ]:
pred_val=Predict(val)
answers, probabilities = model.predict(pred_val,n_best_size=5)

print(answers[10])
# print(probabilities[10])

Running Prediction:   0%|          | 0/4 [00:00<?, ?it/s]

{'id': '9:12-16_400', 'answer': ['قاتلوهم يعذبهم الله بأيديكم ويخزهم وينصركم عليهم ويشف صدور قوم مؤمنين', 'وإن نكثوا أيمانهم من بعد عهدهم وطعنوا في دينكم فقاتلوا أئمة الكفر إنهم لا أيمان لهم لعلهم ينتهون. ألا تقاتلون قوما نكثوا أيمانهم وهموا بإخراج الرسول وهم بدءوكم أول مرة أتخشونهم فالله أحق أن تخشوه إن كنتم مؤمنين. قاتلوهم يعذبهم الله بأيديكم ويخزهم وينصركم عليهم ويشف صدور قوم مؤمنين', 'وإن نكثوا أيمانهم من بعد عهدهم وطعنوا في دينكم فقاتلوا أئمة الكفر', 'قاتل', 'ين']}


# Seventh:Submission Preparation

-prepare the data format

-convert data to json file

##Preparing The Data Format


Example for record in answers

{'id': '9:12-16_400', 'answer': ['قاتلوهم يعذبهم الله بأيديكم ويخزهم وينصركم عليهم ويشف صدور قوم مؤمنين', 'وإن نكثوا أيمانهم من بعد عهدهم وطعنوا في دينكم فقاتلوا أئمة الكفر إنهم لا أيمان لهم لعلهم ينتهون. ألا تقاتلون قوما نكثوا أيمانهم وهموا بإخراج الرسول وهم بدءوكم أول مرة أتخشونهم فالله أحق أن تخشوه إن كنتم مؤمنين. قاتلوهم يعذبهم الله بأيديكم ويخزهم وينصركم عليهم ويشف صدور قوم مؤمنين', 'قاتل', 'قاتلوهم يعذبهم الله بأيديكم ويخزهم وينصركم عليهم', 'قاتلوهم يعذبهم الله بأيديكم ويخزهم وينصركم عليهم ويشف صدور قوم مؤمنين. ويذهب غيظ قلوبهم ويتوب الله على من يشاء والله عليم حكيم. أم حسبتم أن تتركوا ولما يعلم الله الذين جاهدوا منكم ولم يتخذوا من دون الله ولا رسوله ولا المؤمنين وليجة']}


Example for record in probabilities

{'id': '9:12-16_400', 'probability': [0.978582441289257, 0.009298518031500566, 0.006677878508173716, 0.0042863732259504125, 0.00115478887609986]}

In [ ]:
def answer_with_ranks(index,answers,probabilities):
  ranks=[] 
  count=1
  for i in range(len(answers[index]['answer'])): #loop in list of answer in the index(sent in function parameter) of answers{list of dictioniries predicted by the model} 
    # if(probabilities[index]["probability"][i]<0.1):
    # #   break
    # if ( answers[index]['answer'][i]==""):
    #   continue
    dict={
        'answer':answers[index]['answer'][i],
        'rank': count,#rank 
        'score':(probabilities[index]["probability"][i]) #take the corresponding probabilty(score) in the list probability which is predicted by the model
                  }
    ranks.append(dict)
    count=count+1
  return ranks


In [ ]:
def SubmissionFile(answers,probabilities):
  run_submission={}
  for i in range(len(answers)):#loop in list of dictioniries answers which contain id of record and 5 answers predicted by the model
    run_submission.update({str(answers[i]['id']) : answer_with_ranks(i,answers,probabilities)})
  return(run_submission)
run_submission=SubmissionFile(answers,probabilities)
print(run_submission)

## Convert Data into Json

In [ ]:
def convert_to_json(run,text):
  with open(text+".json", "w", encoding='utf-8') as final:
    json.dump(obj=run,fp=final, indent=1,ensure_ascii=False)
File='augbert_05'
convert_to_json(run_submission,File)

#Eighth:The Competition Scripts

## Submission Checker Script

In [ ]:
!cp /content/drive/MyDrive/AraElectra/quranqa/code/quranqa22_submission_checker.py .
!python quranqa22_submission_checker.py\
    --run_file="/content/drive/MyDrive/AraElectra/augbert_05.json"

The run file is correct.


## Evaluation Checker Script

In [13]:
!cp /content/drive/MyDrive/AraElectra/quranqa/code/quranqa22_eval.py .
!python quranqa22_eval.py \
  --gold_answers_file "/content/drive/MyDrive/AraElectra/quranqa/datasets/qrcd_v1.1_dev.jsonl" \
  --run_file "/content/drive/MyDrive/AraElectra/augbert_05.json"

[2022-06-19 14:28:37,060 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from /content/drive/MyDrive/AraElectra/quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.5403968125752556, "exact_match": 0.3394495412844037, "f1": 0.5274263953044676}


#Ninth:Testing

##predict

In [ ]:
pred_test=Predict(test)
test_answers, test_probabilities = model.predict(pred_test,n_best_size=5)

add example index and unique id: 100%|██████████| 274/274 [00:00<00:00, 111274.14it/s]


Running Prediction:   0%|          | 0/9 [00:00<?, ?it/s]

##Submission file

In [ ]:
test_submission=SubmissionFile(test_answers,test_probabilities)
# test_submission

##convert file to json 

In [ ]:
convert_to_json(test_submission,"bertfin_01")

##Submission checker

In [ ]:
!python quranqa22_submission_checker.py\
    --run_file="/content/drive/MyDrive/AraElectra/bertfin_01.json"

The run file is correct.


#Tenth : Human Judgement

In [ ]:
with open(r"bertfin_01.json", "r") as read_file:
    best = json.load(read_file)
#best

In [ ]:
import random
indexes=[]
for i in range(100):
  while True:#because if the random number exist in ids exist we will make a new random
      new_index=random.randint(0,len(test))# generate random number
      if new_index not in indexes:# if the generated random number not in indexes list
        indexes.append(new_index)
        break

len(indexes)

100

In [ ]:
Id=[]
MainContext=[]
Questions=[]
Rank1Answers=[]
Rank2Answers=[]
for i in indexes:
  Id.append(test[i]["qas"][0]['id'])
  MainContext.append(test[i]['context'])
  Questions.append(test[i]['qas'][0]['question'])
  Rank1Answers.append(best[test[i]["qas"][0]['id']][0]["answer"])
# print(Id)
# print(MainContext)
# print(Questions[0])
# print(Rank1Answers[0])

##Converting to csv

In [ ]:
df = pd.DataFrame(list(zip(Id,MainContext,Questions,Rank1Answers)),columns =['Id','MainContext','Question','Rank1Answer'])
df.to_csv('Augmented AraBERT Model Human Judgement.csv',index=False,header=True,encoding="UTF-8")